### Вариант 1
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию).  
Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:  
Наименование вакансии.  
Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
Ссылку на саму вакансию.  
Сайт, откуда собрана вакансия.  
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [ ]:
import json
import requests
import pprint
from bs4 import BeautifulSoup as bs
import re
from time import sleep
import pandas as pd
import numpy as np

### Список ссылок
Найдем все страницы с интересующим нас запросом исоздадим список ссылок для дальнейшего использования

In [439]:
def search_vacancy():
    

    """ Функция принимает запрос на вакансию от пользователяи возвращает список стр 
    """
    
    from bs4 import BeautifulSoup as bs
    from requests import get
    # Сборка url
    vacancy_name = input('Введите название вакансии - ')
    url = r'https://hh.ru/search/vacancy?'

    param = {'text': vacancy_name,\
             'salary':'', \
             'ored_clusters':'true', \
             'items_on_page':'20', \
             'enable_snippets':'true', \
             'only_with_salary': 'true', \
             'page':''}
    
    headers = {'User-Agent': 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    
    response = get(url, params=param, headers=headers)
    ful_url = response.url

    if response.ok:
        soup = bs(response.content, 'lxml')
        result_of_requie = soup.select_one('h1.bloko-header-section-3').text.split(' ')# список результатов по запросу
        if result_of_requie[0] =='По':
            return f'По запросу {vacancy_name} ничего не найдено'
        else:
            number_of_vac = int(result_of_requie[0].replace(u'\xa0', u'')) # Замена разделителя \xa в 1м элементе списка (содержит число вакансии)
            number_of_pages = int(round(number_of_vac/20,0)-1) # выесняем чтсло стр (на 1 стр 50 вакансий)
            list_of_urls = [ful_url + str(i) for i in range(number_of_pages)]
    return list_of_urls

list_of_links = search_vacancy()
list_of_links



Введите название вакансии - data science


['https://vladimir.hh.ru/search/vacancy?text=data+science&salary=&ored_clusters=true&items_on_page=20&enable_snippets=true&only_with_salary=true&page=0',
 'https://vladimir.hh.ru/search/vacancy?text=data+science&salary=&ored_clusters=true&items_on_page=20&enable_snippets=true&only_with_salary=true&page=1',
 'https://vladimir.hh.ru/search/vacancy?text=data+science&salary=&ored_clusters=true&items_on_page=20&enable_snippets=true&only_with_salary=true&page=2',
 'https://vladimir.hh.ru/search/vacancy?text=data+science&salary=&ored_clusters=true&items_on_page=20&enable_snippets=true&only_with_salary=true&page=3',
 'https://vladimir.hh.ru/search/vacancy?text=data+science&salary=&ored_clusters=true&items_on_page=20&enable_snippets=true&only_with_salary=true&page=4',
 'https://vladimir.hh.ru/search/vacancy?text=data+science&salary=&ored_clusters=true&items_on_page=20&enable_snippets=true&only_with_salary=true&page=5',
 'https://vladimir.hh.ru/search/vacancy?text=data+science&salary=&ored_clust

### Сбор Информации
Собираем информацию со всех страниц и возвращаем джсон с 4 мя полями name, money, links, location

In [517]:
def parser_info(list_urls):
    
    """ 
    """
    from bs4 import BeautifulSoup as bs
    import requests
    headers = {'User-Agent': 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    
    
    name =[]
    money =[]
    links = []
    location = []
    vacancy_dict = {}
    for site in list_urls:
        response=requests.get(site, headers=headers)
        sleep(3)
        soup = bs(response.content, 'lxml')
        
        for el in soup.select('a.serp-item__title'):
            name.append(el.text)
        
        for el in soup.select('span.bloko-header-section-3'):
            money.append(el.text)

        for el in soup.find_all('a', {'class': 'serp-item__title'}):
            link = el.get('href')
            links.append(link)      
        for el in soup.find_all('div', {'data-qa': 'vacancy-serp__vacancy-address'}):
            location.append(el.text)
    vacancy_dict['name'] =  name
    vacancy_dict['money'] = money
    vacancy_dict['links'] = links
    vacancy_dict['location'] = location
    
    
    return vacancy_dict

info_json = parser_info(list_of_links)



### Коректируем данные 
Разносим столбец с money на 3 состовляющих - min, max, currency, возвращаем датафрейм

In [502]:
def correction_data(info_json):
    """ 
    Коректирует поле money bи разносить ео на 3 отдельных столбца    """
   
    min_money = []
    max_money = []
    currency = []
    data_lists = []
    for sentence in info_json['money']:
        sentence = sentence.replace(u"\u202f", u"")
        data_list = re.split(r'\s+|–', sentence)
        data_lists.append(data_list)


    for index, list_of_numbers in enumerate(data_lists):
        if list_of_numbers[0] == 'от':
            min_money.append(None)
            max_money.append(int(list_of_numbers[1]))
            currency.append(list_of_numbers[2])
        elif list_of_numbers[0] == 'до':
            min_money.append(int(list_of_numbers[1]))
            max_money.append('None')
            currency.append(list_of_numbers[2])
        else:
            min_money.append(int(list_of_numbers[0]))
            max_money.append(int(list_of_numbers[3]))
            currency.append(list_of_numbers[4])
            
    info_json['min_money'] = min_money
    info_json['max_money'] = max_money
    info_json['currency'] = currency
    info_json.pop('money')
    df = pd.DataFrame(info_json)
    df = df.replace(['None'], np.NaN)
    df = df.astype({'max_money': np.float64 })
    
    return df
        
df = correction_data(info_json) 
df

,name,links,location,min_money,max_money,currency
0,Data scientist,https://vladimir.hh.ru/vacancy/74376536?from=v...,Москва,200000.0,240000,руб.
1,Quantitative Analyst,https://vladimir.hh.ru/vacancy/76506452?from=v...,Кипр,5000.0,8000,EUR
2,Machine Learning Engineer,https://vladimir.hh.ru/vacancy/76983946?from=v...,Баку,4000.0,8000,USD
3,Middle/Senior Data Engineer,https://vladimir.hh.ru/vacancy/71220088?from=v...,"Москва, Домодедовская",NaN,200000,руб.
4,Data-engeneer (Middle),https://vladimir.hh.ru/vacancy/76921562?from=v...,Москва,250000.0,300000,руб.
...,...,...,...,...,...,...
175,Team Leader (PHP) / Руководитель группы разраб...,https://vladimir.hh.ru/vacancy/76345172?from=v...,Москва,NaN,350000,руб.
176,Системный администратор,https://vladimir.hh.ru/vacancy/76323647?from=v...,Москва,100000.0,None,руб.
177,Senior PHP Developer,https://vladimir.hh.ru/vacancy/76276201?from=v...,Москва,NaN,5500,EUR
178,Личный ассистент/начинающий маркетолог,https://vladimir.hh.ru/vacancy/76167353?from=v...,"Минск, Академия наук и еще 2",600.0,1500,бел.


In [518]:

df.dtypes

name          object
links         object
location      object
min_money    float64
max_money    float64
currency      object
dtype: object

In [519]:
df.to_csv(r'hh_ru.csv', index= False)